In [ ]:
%matplotlib widget
import scipp as sc
from dataclasses import dataclass
import plopp


@dataclass
class DataHandle:
    value: sc.DataArray | None


class DataProvider:
    def __init__(self, value: sc.DataArray | None):
        self._handle = DataHandle(value)
        self._node = plopp.Node(self._handle)

        @plopp.node
        def get_value(handle: DataHandle) -> sc.DataArray | None:
            return handle.value

        self.value_node = get_value(self._node)

    def update_data(self, value: sc.DataArray):
        self._handle.value = value
        self._node.notify_children(message='hi')


providers: dict[str, DataProvider] = {}
images = [
    plopp.imagefigure(vmin=1, vmax=200, cbar=True, aspect='equal') for _ in range(4)
]

In [ ]:
import asyncio

import msgpack
import websockets

from beamlime.core.serialization import deserialize_data_array


def websocket_worker():
    """Background thread for WebSocket communication"""

    async def connect_websocket():
        uri = "ws://localhost:5555/ws"
        async with websockets.connect(uri) as websocket:
            while True:
                try:
                    data = await websocket.recv()
                    if data:
                        try:
                            arrays = {
                                tuple(k.split('||')): deserialize_data_array(v)
                                for k, v in msgpack.unpackb(data).items()
                            }
                            for i, (key, value) in enumerate(arrays.items()):
                                # plopp.imagefigure needs coords?
                                name = ' '.join(key)
                                for dim in value.dims:
                                    if dim not in value.coords:
                                        value.coords[dim] = sc.arange(
                                            dim, value.sizes[dim], unit=None
                                        )
                                if name not in providers:
                                    providers[name] = DataProvider(value)
                                    providers[name].value_node.add_view(images[i])
                                else:
                                    providers[name].update_data(value)
                        except (msgpack.UnpackException, KeyError) as e:
                            print("Error processing data: %s", e)
                except websockets.ConnectionClosed:
                    print("WebSocket connection closed, attempting to reconnect...")
                    await asyncio.sleep(2)
                    break

    async def keep_alive():
        while True:
            try:
                await connect_websocket()
            except Exception as e:
                print(f"WebSocket error: {e}")
                await asyncio.sleep(2)

    asyncio.run(keep_alive())

In [ ]:
import ipywidgets as ipw

row1 = images[0:2]
row2 = images[2:4]
ipw.VBox([ipw.HBox(row1), ipw.HBox(row2)])

In [ ]:
import threading

threading.Thread(target=websocket_worker, daemon=True).start()